<a href="https://colab.research.google.com/github/LeeSeungwon89/Kaggle_Dacon_Practice/blob/main/1_Bike_Sharing_Demand_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle
from google.colab import files
files.upload()

In [3]:
ls -1ha kaggle.json

kaggle.json


In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# Permission Warning이 발생하지 않도록 해줍니다.
!chmod 600 ~/.kaggle/kaggle.json

# 참가한 대회 리스트를 확인합니다.
!kaggle competitions list

In [5]:
!kaggle competitions download -c bike-sharing-demand

  0% 0.00/189k [00:00<?, ?B/s]
100% 189k/189k [00:00<00:00, 58.7MB/s]


In [6]:
!ls

bike-sharing-demand.zip  kaggle.json  sample_data


In [7]:
!unzip bike-sharing-demand.zip

Archive:  bike-sharing-demand.zip
  inflating: sampleSubmission.csv    
  inflating: test.csv                
  inflating: train.csv               


# **1. 데이터 준비**

데이터를 준비하겠습니다.

In [103]:
import numpy as np
import pandas as pd
import random

np.random.seed(2022)
random.seed(2022)

# 최대 행렬 수를 설정합니다.
pd.set_option('display.max_column', 50)
pd.set_option('display.max_rows', 50)

# 데이터를 읽습니다.
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submission = pd.read_csv('sampleSubmission.csv')

# **2. 데이터 확인**

데이터를 살펴보겠습니다.

## **2.1. 데이터 개요**

훈련 세트를 확인하겠습니다.

In [104]:
train.head(3)

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32


In [105]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   datetime    10886 non-null  object 
 1   season      10886 non-null  int64  
 2   holiday     10886 non-null  int64  
 3   workingday  10886 non-null  int64  
 4   weather     10886 non-null  int64  
 5   temp        10886 non-null  float64
 6   atemp       10886 non-null  float64
 7   humidity    10886 non-null  int64  
 8   windspeed   10886 non-null  float64
 9   casual      10886 non-null  int64  
 10  registered  10886 non-null  int64  
 11  count       10886 non-null  int64  
dtypes: float64(3), int64(8), object(1)
memory usage: 1020.7+ KB


In [106]:
train.describe()

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
count,10886.000000,10886.000000,10886.000000,10886.000000,10886.00000,10886.000000,10886.000000,10886.000000,10886.000000,10886.000000,10886.000000
mean,2.506614,0.028569,0.680875,1.418427,20.23086,23.655084,61.886460,12.799395,36.021955,155.552177,191.574132
std,1.116174,0.166599,0.466159,0.633839,7.79159,8.474601,19.245033,8.164537,49.960477,151.039033,181.144454
min,1.000000,0.000000,0.000000,1.000000,0.82000,0.760000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,2.000000,0.000000,0.000000,1.000000,13.94000,16.665000,47.000000,7.001500,4.000000,36.000000,42.000000
50%,3.000000,0.000000,1.000000,1.000000,20.50000,24.240000,62.000000,12.998000,17.000000,118.000000,145.000000
75%,4.000000,0.000000,1.000000,2.000000,26.24000,31.060000,77.000000,16.997900,49.000000,222.000000,284.000000
max,4.000000,1.000000,1.000000,4.000000,41.00000,45.455000,100.000000,56.996900,367.000000,886.000000,977.000000


특이사항은 없습니다. 

테스트 세트를 확인하겠습니다.

In [107]:
test.head(3)

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed
0,2011-01-20 00:00:00,1,0,1,1,10.66,11.365,56,26.0027
1,2011-01-20 01:00:00,1,0,1,1,10.66,13.635,56,0.0000
2,2011-01-20 02:00:00,1,0,1,1,10.66,13.635,56,0.0000


In [108]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6493 entries, 0 to 6492
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   datetime    6493 non-null   object 
 1   season      6493 non-null   int64  
 2   holiday     6493 non-null   int64  
 3   workingday  6493 non-null   int64  
 4   weather     6493 non-null   int64  
 5   temp        6493 non-null   float64
 6   atemp       6493 non-null   float64
 7   humidity    6493 non-null   int64  
 8   windspeed   6493 non-null   float64
dtypes: float64(3), int64(5), object(1)
memory usage: 456.7+ KB


In [109]:
test.describe()

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed
count,6493.000000,6493.000000,6493.000000,6493.000000,6493.000000,6493.000000,6493.000000,6493.000000
mean,2.493300,0.029108,0.685815,1.436778,20.620607,24.012865,64.125212,12.631157
std,1.091258,0.168123,0.464226,0.648390,8.059583,8.782741,19.293391,8.250151
min,1.000000,0.000000,0.000000,1.000000,0.820000,0.000000,16.000000,0.000000
25%,2.000000,0.000000,0.000000,1.000000,13.940000,16.665000,49.000000,7.001500
50%,3.000000,0.000000,1.000000,1.000000,21.320000,25.000000,65.000000,11.001400
75%,3.000000,0.000000,1.000000,2.000000,27.060000,31.060000,81.000000,16.997900
max,4.000000,1.000000,1.000000,4.000000,40.180000,50.000000,100.000000,55.998600


훈련 세트와 달리 'casual', 'registered', 'count' 피처가 없습니다. 'count' 피처는 타깃값을 나타냅니다.

## **2.2. 피처 개요**

피처를 하나씩 살펴보겠습니다.

### **2.2.1. 피처 요약**

피처 요약표를 토대로 세부 사항을 확인하겠습니다.

In [110]:
def feature_table(df):
    # 피처 이름과 타입을 출력합니다.     
    feature_description = pd.DataFrame(df.dtypes, columns=['type'])
    feature_description = feature_description.reset_index()
    feature_description = feature_description.rename(columns={'index': 'feature'})
    # 결측치 개수를 산출합니다.
    feature_description['null count'] = df.isnull().sum().values
    # 고윳값 개수를 산출합니다.
    feature_description['eigenvalue count'] = df.nunique().values
    
    # 데이터 크기를 출력합니다.
    print(df.shape)
    
    return feature_description

In [111]:
feature_table(train)

(10886, 12)


,feature,type,null count,eigenvalue count
0,datetime,object,0,10886
1,season,int64,0,4
2,holiday,int64,0,2
3,workingday,int64,0,2
4,weather,int64,0,4
5,temp,float64,0,49
6,atemp,float64,0,60
7,humidity,int64,0,89
8,windspeed,float64,0,28
9,casual,int64,0,309


In [112]:
feature_table(test)

(6493, 9)


,feature,type,null count,eigenvalue count
0,datetime,object,0,6493
1,season,int64,0,4
2,holiday,int64,0,2
3,workingday,int64,0,2
4,weather,int64,0,4
5,temp,float64,0,49
6,atemp,float64,0,65
7,humidity,int64,0,79
8,windspeed,float64,0,27


- 'count' 피처는 타깃값입니다. 모델링을 수행하기 전에 훈련 세트에서 y값으로 분리해야 합니다.

- 결측치는 없습니다.

- 훈련 세트와 달리 테스트 세트에는 'casual', 'registered', 'count' 피처가 없습니다. 'casual', 'registered' 피처는 추후 모델링을 수행할 때 훈련 세트에서 제거해야 합니다.

## **2.3. 피처 전처리**

데이터를 시각화하기 전에 전처리를 수행할 피처들을 살펴 보겠습니다.

### **2.3.1. datetime**

'datetime' 피처의 타입은 `object`입니다. 문자열을 나타냅니다. 

In [113]:
train['datetime'][0]

'2011-01-01 00:00:00'

문자열은 시각화에 효율적으로 적용할 수 없습니다. 연, 월, 일, 시, 분, 초로 나누어 각 요소마다 새로운 피처를 생성해야 합니다. 이 과정을 수행하겠습니다.

In [114]:
train['year'] = train['datetime'].apply(lambda x: x.split()[0].split('-')[0])
train['month'] = train['datetime'].apply(lambda x: x.split()[0].split('-')[1])
train['day'] = train['datetime'].apply(lambda x: x.split()[0].split('-')[2])
train['hour'] = train['datetime'].apply(lambda x: x.split()[1].split(':')[0])
train['minute'] = train['datetime'].apply(lambda x: x.split()[1].split(':')[1])
train['second'] = train['datetime'].apply(lambda x: x.split()[1].split(':')[2])

In [115]:
train.head(3)

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,year,month,day,hour,minute,second
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16,2011,01,01,00,00,00
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40,2011,01,01,01,00,00
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32,2011,01,01,02,00,00


추가로 요일 피처도 생성해 보겠습니다.

In [116]:
from datetime import datetime
import calendar

# 날짜만 추출합니다.
train['date'] = train['datetime'].apply(lambda x: x.split()[0])

# 날짜에 해당하는 숫자를 추출하여 요일명으로 치환합니다.
train['day_of_week'] = train['date'].apply(lambda x: calendar.day_name
                                           [datetime.strptime(x, '%Y-%m-%d').weekday()])

train['day_of_week'].value_counts()

Saturday     1584
Sunday       1579
Thursday     1553
Monday       1551
Wednesday    1551
Tuesday      1539
Friday       1529
Name: day_of_week, dtype: int64

### **2.3.2. season**

'season' 피처는 정수 레이블로 구성된 명목형 피처입니다. 

- 1: spring

- 2: summer

- 3: fall

- 4: winter 

In [117]:
train['season'].value_counts()

4    2734
2    2733
3    2733
1    2686
Name: season, dtype: int64

그대로 사용해도 괜찮지만 구분하기 쉽도록 각 값에 해당하는 계절로 치환하겠습니다.

In [118]:
train['season'] = train['season'].map({
    1: 'spring',
    2: 'summer',
    3: 'fall',
    4: 'winter'
})

train['season'].value_counts()

winter    2734
summer    2733
fall      2733
spring    2686
Name: season, dtype: int64

### **2.3.3. holiday**

'holiday' 피처는 정수 레이블로 구성된 명목형 피처입니다.

- 0: 공휴일 아님

- 1: 공휴일

레이블 수가 적으므로 구분하기 쉽습니다. 그대로 사용하겠습니다.

### **2.3.4. workingday**

'workingday' 피처는 정수 레이블로 구성된 명목형 피처입니다.

- 0: 근로일 아님

- 1: 근로일

레이블 수가 적으므로 구분하기 쉽습니다. 그대로 사용하겠습니다.

### **2.3.5. weather**

'weather' 피처는 정수 레이블로 구성된 명목형 피처입니다.

- 1: Clear, Few clouds, Partly cloudy, Partly cloudy

- 2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist

- 3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds

- 4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog

In [119]:
train['weather'].value_counts()

1    7192
2    2834
3     859
4       1
Name: weather, dtype: int64

구분하기 쉽도록 각 값에 해당하는 날씨로 치환하겠습니다.

In [120]:
train['weather'] = train['weather'].map({
    1: 'clear',
    2: 'mist',
    3: 'light',
    4: 'bad'
})

train['weather'].value_counts()

clear    7192
mist     2834
light     859
bad         1
Name: weather, dtype: int64

### **2.3.6. temp**

'temp'는 연속형 피처이며, 실제 온도를 의미합니다. 전처리를 수행할 필요가 없습니다.

In [128]:
train['temp'].describe()

count    10886.00000
mean        20.23086
std          7.79159
min          0.82000
25%         13.94000
50%         20.50000
75%         26.24000
max         41.00000
Name: temp, dtype: float64

### **2.3.7. atemp**

'atemp'는 연속형 피처이며, 체감 온도를 의미합니다. 전처리를 수행할 필요가 없습니다.

In [129]:
train['atemp'].describe()

count    10886.000000
mean        23.655084
std          8.474601
min          0.760000
25%         16.665000
50%         24.240000
75%         31.060000
max         45.455000
Name: atemp, dtype: float64

### **2.3.8. humiduty**

'humidity'는 이산형 피처이며, 상대 습도를 의미합니다. 전처리를 수행할 필요가 없습니다.

In [130]:
train['humidity'].describe()

count    10886.000000
mean        61.886460
std         19.245033
min          0.000000
25%         47.000000
50%         62.000000
75%         77.000000
max        100.000000
Name: humidity, dtype: float64

### **2.3.9. windspeed**

'windspeed'는 연속형 피처이며, 풍속을 의미합니다. 전처리를 수행할 필요가 없습니다.

In [134]:
train['windspeed'].describe()

count    10886.000000
mean        12.799395
std          8.164537
min          0.000000
25%          7.001500
50%         12.998000
75%         16.997900
max         56.996900
Name: windspeed, dtype: float64

# **3. 데이터 시각화**

# **4. 결론**



---

업데이트 중

---

